# Test to ensure modifications to function do not change evaluated value

Beginning modification of the function to handle case where N is not equal to 1.

In [1]:
import math
from numba import cuda
import ZMCIntegral
import time
import numpy as np
import scipy
import scipy.special
from scipy.integrate import quad

Define constants in function

In [2]:
mu = 0.1  # Fermi-level
hOmg = 0.5  # Photon energy eV
a = 4  # AA
A = 4  # hbar^2/(2m)=4 evAA^2 (for free electron mass)
rati = 0.3  # ratio
eE0 = rati * ((hOmg) ** 2) / (2 * np.sqrt(A * mu))
# print(eE0)
Gamm = 0.005  # Gamma in eV.
KT = 1 * 10 ** (-6)
shift = A * (eE0 / hOmg) ** 2

The original function is declared below.

In [3]:
def Ds(kx, ky, qx, qy):
    N = 2
    dds = 0
    ds = 0
    ek = A * (math.sqrt((kx) ** 2 + (ky) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    ekq = A * (math.sqrt((kx + qx) ** 2 + (ky + qy) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    xk = 2 * A * eE0 * math.sqrt((kx) ** 2 + (ky) ** 2) / hOmg ** 2
    xkq = 2 * A * eE0 * math.sqrt((kx + qx) ** 2 + (ky + qy) ** 2) / hOmg ** 2
    
    # print('xk = ', xk)
    # print('xkq = ', xkq)

    sing = np.arange(-(N - 1) / 2, (N - 1) / 2 + 1, 1)
    taninv1kp = 2 * np.arctan2(Gamm, ek - hOmg / 2 + hOmg * sing)
    taninv1kqp = 2 * np.arctan2(Gamm, ekq - hOmg / 2 + hOmg * sing)
    taninv1km = 2 * np.arctan2(Gamm, ek + hOmg / 2 + hOmg * sing)
    taninv1kqm = 2 * np.arctan2(Gamm, ekq + hOmg / 2 + hOmg * sing)

    lg1kp = complex(0, 1) * np.log(Gamm ** 2 + (ek - hOmg / 2 + hOmg * sing) ** 2)
    lg1kqp = complex(0, 1) * np.log(Gamm ** 2 + (ekq - hOmg / 2 + hOmg * sing) ** 2)
    lg1km = complex(0, 1) * np.log(Gamm ** 2 + (ek + hOmg / 2 + hOmg * sing) ** 2)
    lg1kqm = complex(0, 1) * np.log(Gamm ** 2 + (ekq + hOmg / 2 + hOmg * sing) ** 2)

    ferp = np.heaviside(mu - hOmg / 2 - hOmg * sing, 0)
    ferm = np.heaviside(mu + hOmg / 2 - hOmg * sing, 0)

    dbl = np.arange(-(N - 1), (N - 1) + 1, 1)
    taninv2k = 2 * np.arctan2(Gamm, ek - mu + hOmg * dbl)
    taninv2kq = 2 * np.arctan2(Gamm, ekq - mu + hOmg * dbl)

    lg2k = complex(0, 1) * np.log(Gamm ** 2 + (ek - mu + hOmg * dbl) ** 2)
    lg2kq = complex(0, 1) * np.log(Gamm ** 2 + (ekq - mu + hOmg * dbl) ** 2)

    besk = scipy.special.jv(dbl, xk)
    beskq = scipy.special.jv(dbl, xkq)

    fac1 = ek - ekq + hOmg * dbl
    fac2 = fac1 + 2 * complex(0, 1) * Gamm
    fac3 = fac2 - ek + ekq
    
    # DEBUGGING
    # print('taninv2k = ', taninv2k)
    # print('taninv1kp = ', taninv1kp)
    # print('lg2k = ', lg2k)
    # print('lg1kp = ', lg1kp)
    # print('fac3 = ', fac3)
    # print('besk = ', besk)
    # print('beskq = ', beskq)
    
    for n in range(0, N):
        for alpha in range(0, N):
            for beta in range(0, N):
                for gamma in range(0, N):
                    for s in range(0, N):
                        for l in range(0, N):
                            p1p = fac1[beta - gamma + N - 1] * (
                                    taninv1kp[alpha] - taninv2k[s + alpha] - lg1kp[alpha] + lg2k[s + alpha])
                            p2p = fac2[alpha - gamma + N - 1] * (
                                    taninv1kp[beta] - taninv2k[s + beta] + lg1kp[beta] - lg2k[s + beta])
                            p3p = fac3[alpha - beta + N - 1] * (
                                    -taninv1kqp[gamma] + taninv2kq[s + gamma] - lg1kqp[gamma] + lg2kq[
                                s + gamma])

                            p1m = fac1[beta - gamma + N - 1] * (
                                    taninv1km[alpha] - taninv2k[s + alpha] - lg1km[alpha] + lg2k[s + alpha])

                            p2m = fac2[alpha - gamma + N - 1] * (
                                    taninv1km[beta] - taninv2k[s + beta] + lg1km[beta] - lg2k[s + beta])

                            p3m = fac3[alpha - beta + N - 1] * (
                                    -taninv1kqm[gamma] + taninv2kq[s + gamma] - lg1kqm[gamma] + lg2kq[
                                s + gamma])

                            d1 = -2 * complex(0, 1) * fac1[beta - gamma + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     alpha - beta + N - 1]
                            
                            omint1p = ferp[s] * ((p1p + p2p + p3p) / d1)

                            omint1m = ferm[s] * ((p1m + p2m + p3m) / d1)

                            bess1 = beskq[gamma - n + N - 1] * beskq[gamma - l + N - 1] * besk[beta - l + N - 1] * besk[
                                beta - s + N - 1] * besk[alpha - s + N - 1] * besk[alpha - n + N - 1]

                            grgl = bess1 * (omint1p - omint1m)

                            pp1p = fac1[alpha - beta + N - 1] * (
                                    -taninv1kqp[gamma] + taninv2kq[s + gamma] - lg1kqp[gamma] + lg2kq[
                                s + gamma])

                            pp2p = fac2[alpha - gamma + N - 1] * (
                                    -taninv1kqp[beta] + taninv2kq[s + beta] + lg1kqp[beta] - lg2kq[
                                s + beta])

                            pp3p = fac3[beta - gamma + N - 1] * (
                                    taninv1kp[alpha] - taninv2k[s + alpha] - lg1kp[alpha] + lg2k[s + alpha])

                            pp1m = fac1[alpha - beta + N - 1] * (
                                    -taninv1kqm[gamma] + taninv2kq[s + gamma] - lg1kqm[gamma] + lg2kq[
                                s + gamma])

                            pp2m = fac2[alpha - gamma + N - 1] * (
                                    -taninv1kqm[beta] + taninv2kq[s + beta] + lg1kqm[beta] - lg2kq[
                                s + beta])

                            pp3m = fac3[beta - gamma + N - 1] * (
                                    taninv1km[alpha] - taninv2k[s + alpha] - lg1km[alpha] + lg2k[s + alpha])

                            d2 = -2 * complex(0, 1) * fac1[alpha - beta + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     beta - gamma + N - 1]

                            omint2p = ferp[s] * ((pp1p + pp2p + pp3p) / d2)

                            omint2m = ferm[s] * ((pp1m + pp2m + pp3m) / d2)

                            bess2 = beskq[gamma - n + N - 1] * beskq[gamma - s + N - 1] * beskq[beta - s + N - 1] * \
                                    beskq[beta - l + N - 1] * besk[alpha - l + N - 1] * besk[alpha - n + N - 1]

                            glga = bess2 * (omint2p - omint2m)

                            dds = dds + 2 * Gamm * (grgl + glga)
                            #DEBUG
                            # print('dds = ', dds)
                            # print('bess1=',  bess1)
    return dds

In [13]:
def my_Besselv(v, z):
    # WILL NOT WORK IF v IS NOT AN INTEGER
    # Conditional to handle case of negative v.
    if(v < 0):
        v = abs(v)
        resultsign = (-1) ** v
    else:
        resultsign = 1
    result = 0    
    # Loop to construct Bessel series sum.
    for n in range(0,20):
        sign = (-1)**n
        exp = 2 * n + v
        term = z ** exp
        r = n + v + 1
        if(r == 0):
            r = 1e-15
        denom = math.gamma(r)
        denom = denom * math.factorial(n)
        denom = denom * (2 ** exp)
        term = term / denom * sign
        # print('for ', n, ': ',term)
        result = result + term
        
    return result * resultsign
        
    
def myHeaviside(z):
    # Wrote this Heaviside expression with it cast in cuda to avoid error below.
    if z <= 0 :
	    return 0
    else :
	    return 1
    
    
def modDsN2(x):
    N = 2
    dds = 0
    ds = 0
    ek = A * (math.sqrt((x[0]) ** 2 + (x[1]) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    ekq = A * (math.sqrt((x[0] + x[2]) ** 2 + (x[1] + 0) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2
    xk = 2 * A * eE0 * math.sqrt((x[0]) ** 2 + (x[1]) ** 2) / hOmg ** 2
    xkq = 2 * A * eE0 * math.sqrt((x[0] + x[2]) ** 2 + (x[1] + 0) ** 2) / hOmg ** 2

    # print('modxk = ', xk)
    # print('modxkq = ', xkq)
    
    j = 0
    i = -(N - 1) / 2
    
    taninv1kp = np.empty(N)
    taninv1kqp = np.empty(N)
    taninv1km = np.empty(N)
    taninv1kqm = np.empty(N)

    lg1kp = np.empty(N, dtype='complex')
    lg1kqp = np.empty(N, dtype='complex')
    lg1km = np.empty(N, dtype='complex')
    lg1kqm = np.empty(N, dtype='complex')

    ferp = np.empty(N)
    ferm = np.empty(N)
    
    while(i < ((N - 1) / 2 + 1)):
        taninv1kp[j] = 2 * math.atan2(Gamm, ek - hOmg / 2 + hOmg * i)
        taninv1kqp[j] = 2 * math.atan2(Gamm, ekq - hOmg / 2 + hOmg * i)
        taninv1km[j] = 2 * math.atan2(Gamm, ek + hOmg / 2 + hOmg * i)
        taninv1kqm[j] = 2 * math.atan2(Gamm, ekq + hOmg / 2 + hOmg * i)

        lg1kp[j] = complex(0, 1) * math.log(Gamm ** 2 + (ek - hOmg / 2 + hOmg * i) ** 2)
        lg1kqp[j] = complex(0, 1) * math.log(Gamm ** 2 + (ekq - hOmg / 2 + hOmg * i) ** 2)
        lg1km[j] = complex(0, 1) * math.log(Gamm ** 2 + (ek + hOmg / 2 + hOmg * i) ** 2)
        lg1kqm[j] = complex(0, 1) * math.log(Gamm ** 2 + (ekq + hOmg / 2 + hOmg * i) ** 2)

        ferp[j] = myHeaviside(mu - hOmg / 2 - hOmg * i)
        ferm[j] = myHeaviside(mu + hOmg / 2 - hOmg * i)
        j = j + 1
        i = i + 1
        
    size_dbl = 2 * N - 1
    
    taninv2k = np.empty(size_dbl)
    taninv2kq = np.empty(size_dbl)

    lg2k = np.empty(size_dbl, dtype='complex')
    lg2kq = np.empty(size_dbl, dtype='complex')

    besk = np.empty(size_dbl)
    beskq = np.empty(size_dbl)

    fac1 = np.empty(size_dbl)
    fac2 = np.empty(size_dbl, dtype='complex')
    fac3 = np.empty(size_dbl, dtype='complex')

    j = 0
    for i in range(-(N - 1), N, 1):
        taninv2k[j] = 2 * math.atan2(Gamm, ek - mu + hOmg * i)
        taninv2kq[j] = 2 * math.atan2(Gamm, ekq - mu + hOmg * i)

        lg2k[j] = complex(0, 1) * math.log(Gamm ** 2 + (ek - mu + hOmg * i) ** 2)
        lg2kq[j] = complex(0, 1) * math.log(Gamm ** 2 + (ekq - mu + hOmg * i) ** 2)

        besk[j] = my_Besselv(i, xk)
        beskq[j] = my_Besselv(i, xkq)

        fac1[j] = ek - ekq + hOmg * i
        fac2[j] = fac1[j] + 2 * complex(0, 1) * Gamm
        fac3[j] = fac2[j] - ek + ekq
        j = j + 1
    
    # debug statements
    # print('taninv2k = ', taninv2k)
    # print('taninv1kp = ', taninv1kp)
    # print('lg2k = ', lg2k)
    # print('lg1kp = ', lg1kp)
    # print('fac3 = ', fac3)
    # print('modbesk = ', besk)
    # print('modbeskq = ', beskq)
    
    for n in range(0, N):
        for alpha in range(0, N):
            for beta in range(0, N):
                for gamma in range(0, N):
                    for s in range(0, N):
                        for l in range(0, N):
                            p1p = fac1[beta - gamma + N - 1] * (
                                    taninv1kp[alpha] - taninv2k[s + alpha] - lg1kp[alpha] + lg2k[s + alpha])
                            p2p = fac2[alpha - gamma + N - 1] * (
                                    taninv1kp[beta] - taninv2k[s + beta] + lg1kp[beta] - lg2k[s + beta])
                            p3p = fac3[alpha - beta + N - 1] * (
                                    -taninv1kqp[gamma] + taninv2kq[s + gamma] - lg1kqp[gamma] + lg2kq[
                                s + gamma])

                            p1m = fac1[beta - gamma + N - 1] * (
                                    taninv1km[alpha] - taninv2k[s + alpha] - lg1km[alpha] + lg2k[s + alpha])

                            p2m = fac2[alpha - gamma + N - 1] * (
                                    taninv1km[beta] - taninv2k[s + beta] + lg1km[beta] - lg2k[s + beta])

                            p3m = fac3[alpha - beta + N - 1] * (
                                    -taninv1kqm[gamma] + taninv2kq[s + gamma] - lg1kqm[gamma] + lg2kq[
                                s + gamma])

                            d1 = -2 * complex(0, 1) * fac1[beta - gamma + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     alpha - beta + N - 1]

                            omint1p = ferp[s] * ((p1p + p2p + p3p) / d1)

                            omint1m = ferm[s] * ((p1m + p2m + p3m) / d1)

                            bess1 = beskq[gamma - n + N - 1] * beskq[gamma - l + N - 1] * besk[beta - l + N - 1] * besk[
                                beta - s + N - 1] * besk[alpha - s + N - 1] * besk[alpha - n + N - 1]

                            grgl = bess1 * (omint1p - omint1m)

                            pp1p = fac1[alpha - beta + N - 1] * (
                                    -taninv1kqp[gamma] + taninv2kq[s + gamma] - lg1kqp[gamma] + lg2kq[
                                s + gamma])

                            pp2p = fac2[alpha - gamma + N - 1] * (
                                    -taninv1kqp[beta] + taninv2kq[s + beta] + lg1kqp[beta] - lg2kq[
                                s + beta])

                            pp3p = fac3[beta - gamma + N - 1] * (
                                    taninv1kp[alpha] - taninv2k[s + alpha] - lg1kp[alpha] + lg2k[s + alpha])

                            pp1m = fac1[alpha - beta + N - 1] * (
                                    -taninv1kqm[gamma] + taninv2kq[s + gamma] - lg1kqm[gamma] + lg2kq[
                                s + gamma])

                            pp2m = fac2[alpha - gamma + N - 1] * (
                                    -taninv1kqm[beta] + taninv2kq[s + beta] + lg1kqm[beta] - lg2kq[
                                s + beta])

                            pp3m = fac3[beta - gamma + N - 1] * (
                                    taninv1km[alpha] - taninv2k[s + alpha] - lg1km[alpha] + lg2k[s + alpha])

                            d2 = -2 * complex(0, 1) * fac1[alpha - beta + N - 1] * fac2[alpha - gamma + N - 1] * \
                                 fac3[
                                     beta - gamma + N - 1]

                            omint2p = ferp[s] * ((pp1p + pp2p + pp3p) / d2)

                            omint2m = ferm[s] * ((pp1m + pp2m + pp3m) / d2)

                            bess2 = beskq[gamma - n + N - 1] * beskq[gamma - s + N - 1] * beskq[beta - s + N - 1] * \
                                    beskq[beta - l + N - 1] * besk[alpha - l + N - 1] * besk[alpha - n + N - 1]

                            glga = bess2 * (omint2p - omint2m)

                            dds = dds + 2 * Gamm * (grgl + glga)
                            
                            #DEBUG
                            # print('moddds = ', dds)
                            # print('modbess1=',  bess1)
    return dds.real

# The modified function

The function is vectorized again to increase efficiency.

The custom Bessel function should now handle any-order*, first-kind Bessel function. *(Integer-order!)

NOTE: machine epsilon value ~E-16 implies that a relative error below E-15 is nonsense. The limit rests around here.

In [16]:
# Make error array.
relerror = np.zeros(900)

print('Comparing modified version and original')
print('================================================================================================')
print(' kx  | ky  | qx  | qy  | Ds          | modDs  ')
print('================================================================================================')
for i in range(0, 75, 5):
    for j in range(0, 75, 5):
        for k in range(1, 4, 1):
            xin = [i/100, j/100, k/100]
            ds_result = Ds(i/100, j/100, k/100, 0).real
            modds_result = modDsN2(xin)
            print('%2.1f  | %2.1f | %2.1f | 0   |'%(i/100, j/100, k/100), ds_result, ' | ', modds_result)
            relerror[i+j+k-1] = abs((modds_result / ds_result.real) - 1)
            

Comparing modified version and original
 kx  | ky  | qx  | qy  | Ds          | modDs  
0.0  | 0.0 | 0.0 | 0   | 3.0415021965433815  |  3.0415021965433815
0.0  | 0.0 | 0.0 | 0   | 2.9339589836388984  |  2.9339589836388984
0.0  | 0.0 | 0.0 | 0   | 2.688873620594237  |  2.688873620594237
0.0  | 0.1 | 0.0 | 0   | 2.142008792060922  |  2.1420087920609214
0.0  | 0.1 | 0.0 | 0   | 2.0664319973420597  |  2.0664319973420597
0.0  | 0.1 | 0.0 | 0   | 1.871111721173968  |  1.8711117211739676
0.0  | 0.1 | 0.0 | 0   | -1.3808652441360754  |  -1.3808652441360751
0.0  | 0.1 | 0.0 | 0   | -1.7415681495541993  |  -1.741568149554199
0.0  | 0.1 | 0.0 | 0   | -2.392242914732454  |  -2.3922429147324533
0.0  | 0.1 | 0.0 | 0   | 0.25552146222908095  |  0.255521462229081
0.0  | 0.1 | 0.0 | 0   | 0.19174893542059573  |  0.19174893542059568
0.0  | 0.1 | 0.0 | 0   | -0.048208361093106535  |  -0.04820836109310658
0.0  | 0.2 | 0.0 | 0   | 0.04392091018154195  |  0.04392091018154198
0.0  | 0.2 | 0.0 | 0   | -0.01560

0.1  | 0.5 | 0.0 | 0   | -0.022954000868233015  |  -0.022954000868233015
0.1  | 0.5 | 0.0 | 0   | -0.022315367806876176  |  -0.022315367806876176
0.1  | 0.5 | 0.0 | 0   | -0.021654688258530654  |  -0.02165468825853066
0.1  | 0.5 | 0.0 | 0   | -0.008292527056432488  |  -0.008292527056432488
0.1  | 0.5 | 0.0 | 0   | -0.008119367792027377  |  -0.008119367792027377
0.1  | 0.5 | 0.0 | 0   | -0.00793775354360574  |  -0.007937753543605737
0.1  | 0.6 | 0.0 | 0   | -0.0034767987668927634  |  -0.0034767987668927643
0.1  | 0.6 | 0.0 | 0   | -0.0034184881633079847  |  -0.003418488163307984
0.1  | 0.6 | 0.0 | 0   | -0.003356843414344724  |  -0.0033568434143447234
0.1  | 0.6 | 0.0 | 0   | -0.001596649861018037  |  -0.001596649861018037
0.1  | 0.6 | 0.0 | 0   | -0.0015742235301555501  |  -0.0015742235301555501
0.1  | 0.6 | 0.0 | 0   | -0.0015503924353672976  |  -0.0015503924353672976
0.1  | 0.7 | 0.0 | 0   | -0.0007795180780809632  |  -0.0007795180780809629
0.1  | 0.7 | 0.0 | 0   | -0.000770081272298

0.2  | 0.1 | 0.0 | 0   | -2.4848284925817206  |  -2.484828492581719
0.2  | 0.1 | 0.0 | 0   | -2.642966072007154  |  -2.642966072007153
0.2  | 0.1 | 0.0 | 0   | -2.5303290994982848  |  -2.530329099498285
0.2  | 0.1 | 0.0 | 0   | -3.0886560636000775  |  -3.0886560636000757
0.2  | 0.1 | 0.0 | 0   | -3.7221192136229395  |  -3.7221192136229426
0.2  | 0.2 | 0.0 | 0   | -15.714461362907935  |  -15.714461362907926
0.2  | 0.2 | 0.0 | 0   | -79.94214127101228  |  -79.94214127101228
0.2  | 0.2 | 0.0 | 0   | -60.397056705494414  |  -60.397056705494414
0.2  | 0.2 | 0.0 | 0   | -1.4320367044501996  |  -1.4320367044501996
0.2  | 0.2 | 0.0 | 0   | -1.110543502423251  |  -1.1105435024232513
0.2  | 0.2 | 0.0 | 0   | -0.8975322181685855  |  -0.8975322181685855
0.2  | 0.3 | 0.0 | 0   | -0.1729341828784158  |  -0.1729341828784158
0.2  | 0.3 | 0.0 | 0   | -0.15320221212402563  |  -0.15320221212402557
0.2  | 0.3 | 0.0 | 0   | -0.13667879342791467  |  -0.13667879342791464
0.2  | 0.3 | 0.0 | 0   | -0.047678786

0.3  | 0.6 | 0.0 | 0   | -0.00082259518879252  |  -0.0008225951887925199
0.3  | 0.6 | 0.0 | 0   | -0.0007915258086806031  |  -0.0007915258086806033
0.3  | 0.6 | 0.0 | 0   | -0.0007614080083125444  |  -0.0007614080083125443
0.3  | 0.6 | 0.0 | 0   | -0.00045845482098825347  |  -0.00045845482098825336
0.3  | 0.6 | 0.0 | 0   | -0.0004429219316121612  |  -0.00044292193161216164
0.3  | 0.6 | 0.0 | 0   | -0.00042775928824908844  |  -0.0004277592882490887
0.3  | 0.7 | 0.0 | 0   | -0.0002592048675153142  |  -0.000259204867515314
0.3  | 0.7 | 0.0 | 0   | -0.00025124925479013923  |  -0.0002512492547901392
0.3  | 0.7 | 0.0 | 0   | -0.00024343952901381534  |  -0.0002434395290138153
0.3  | 0.7 | 0.0 | 0   | -0.00014832417015212202  |  -0.00014832417015212202
0.3  | 0.7 | 0.0 | 0   | -0.0001441657330816373  |  -0.00014416573308163728
0.3  | 0.7 | 0.0 | 0   | -0.00014006481360659677  |  -0.0001400648136065968
0.4  | 0.0 | 0.0 | 0   | -0.11381374674012133  |  -0.11381374674012133
0.4  | 0.0 | 0.0 | 0  

0.5  | 0.3 | 0.0 | 0   | -0.0020898640215513903  |  -0.00208986402155139
0.5  | 0.3 | 0.0 | 0   | -0.0019583289076365966  |  -0.001958328907636597
0.5  | 0.3 | 0.0 | 0   | -0.0014749071851361602  |  -0.0014749071851361604
0.5  | 0.3 | 0.0 | 0   | -0.0013882566000123617  |  -0.0013882566000123632
0.5  | 0.3 | 0.0 | 0   | -0.0013077091825807092  |  -0.0013077091825807088
0.5  | 0.4 | 0.0 | 0   | -0.000953739291846911  |  -0.0009537392918469104
0.5  | 0.4 | 0.0 | 0   | -0.0009023126646259385  |  -0.0009023126646259381
0.5  | 0.4 | 0.0 | 0   | -0.0008540821636063065  |  -0.0008540821636063065
0.5  | 0.5 | 0.0 | 0   | -0.0006075955314897177  |  -0.0006075955314897177
0.5  | 0.5 | 0.0 | 0   | -0.0005774817224812328  |  -0.000577481722481233
0.5  | 0.5 | 0.0 | 0   | -0.0005490153128131579  |  -0.0005490153128131578
0.5  | 0.5 | 0.0 | 0   | -0.0003831184143299244  |  -0.0003831184143299246
0.5  | 0.5 | 0.0 | 0   | -0.00036562053007341505  |  -0.0003656205300734153
0.5  | 0.5 | 0.0 | 0   | -0.0

0.6  | 0.7 | 0.0 | 0   | -2.6062468015475257e-05  |  -2.606246801547527e-05
0.7  | 0.0 | 0.0 | 0   | -0.0008186119716150023  |  -0.0008186119716150019
0.7  | 0.0 | 0.0 | 0   | -0.0007641357178063921  |  -0.0007641357178063911
0.7  | 0.0 | 0.0 | 0   | -0.0007142289789895223  |  -0.000714228978989522
0.7  | 0.1 | 0.0 | 0   | -0.000796928065284128  |  -0.0007969280652841278
0.7  | 0.1 | 0.0 | 0   | -0.0007441567662398508  |  -0.0007441567662398511
0.7  | 0.1 | 0.0 | 0   | -0.0006957829246426222  |  -0.0006957829246426222
0.7  | 0.1 | 0.0 | 0   | -0.0007358942078828917  |  -0.0007358942078828914
0.7  | 0.1 | 0.0 | 0   | -0.0006878712100102974  |  -0.0006878712100102969
0.7  | 0.1 | 0.0 | 0   | -0.0006437728057549507  |  -0.0006437728057549506
0.7  | 0.1 | 0.0 | 0   | -0.0006461527896923699  |  -0.0006461527896923695
0.7  | 0.1 | 0.0 | 0   | -0.0006049682598379686  |  -0.0006049682598379686
0.7  | 0.1 | 0.0 | 0   | -0.0005670448115969239  |  -0.0005670448115969238
0.7  | 0.2 | 0.0 | 0   | -

In [17]:
errorsum = 0
print('Comparing modified version and original')
print('================================================================================================')
print(' kx  | ky  | qx  | qy  | rel error  ')
print('================================================================================================')
for i in range(0, 75, 5):
    for j in range(0, 75, 5):
        for k in range(1, 4, 1):
            print('%2.1f  | %2.1f | %2.1f | 0   |'%(i/100, j/100, k/100), relerror[i+j+k-1])
            errorsum = errorsum + relerror[i+j+k-1]
           
avgerror = errorsum / (900)            
print('================================================================================================')
print('The average error of modified function is ', avgerror)

Comparing modified version and original
 kx  | ky  | qx  | qy  | rel error  
0.0  | 0.0 | 0.0 | 0   | 0.0
0.0  | 0.0 | 0.0 | 0   | 0.0
0.0  | 0.0 | 0.0 | 0   | 0.0
0.0  | 0.1 | 0.0 | 0   | 3.3306690738754696e-16
0.0  | 0.1 | 0.0 | 0   | 3.3306690738754696e-16
0.0  | 0.1 | 0.0 | 0   | 6.661338147750939e-16
0.0  | 0.1 | 0.0 | 0   | 0.0
0.0  | 0.1 | 0.0 | 0   | 3.3306690738754696e-16
0.0  | 0.1 | 0.0 | 0   | 4.440892098500626e-16
0.0  | 0.1 | 0.0 | 0   | 0.0
0.0  | 0.1 | 0.0 | 0   | 4.440892098500626e-16
0.0  | 0.1 | 0.0 | 0   | 0.0
0.0  | 0.2 | 0.0 | 0   | 2.220446049250313e-16
0.0  | 0.2 | 0.0 | 0   | 1.1102230246251565e-15
0.0  | 0.2 | 0.0 | 0   | 5.551115123125783e-16
0.0  | 0.2 | 0.0 | 0   | 1.3322676295501878e-15
0.0  | 0.2 | 0.0 | 0   | 7.771561172376096e-16
0.0  | 0.2 | 0.0 | 0   | 2.220446049250313e-16
0.0  | 0.3 | 0.0 | 0   | 2.220446049250313e-16
0.0  | 0.3 | 0.0 | 0   | 0.0
0.0  | 0.3 | 0.0 | 0   | 0.0
0.0  | 0.3 | 0.0 | 0   | 0.0
0.0  | 0.3 | 0.0 | 0   | 0.0
0.0  | 0.3 | 0.0 

# Ensure that treating the arrays can be implemented with the loops above

In [7]:
N = 2

sing = np.arange(-(N - 1) / 2, (N - 1) / 2 + 1, 1)
dbl = np.arange(-(N - 1), (N - 1) + 1, 1)

# rng1 = range(-(N - 1) / 2, (N - 1) / 2 + 1, 1)
print('sing:')
print(sing)
print('\nsing as loop:')
k = -(N - 1) / 2    
while(k < ((N - 1) / 2 + 1)):
    print(k)
    k = k + 1
        
print('\ndbl:')
print(dbl)
print('\ndbl as range:')
for i in range(-(N - 1), N, 1):
    print(i)

sing:
[-0.5  0.5]

sing as loop:
-0.5
0.5

dbl:
[-1  0  1]

dbl as range:
-1
0
1


# Checking the consistency of math and numpy log function

In [8]:
A = 4  # hbar^2/(2m)=4 evAA^2 (for free electron mass)
eE0 = rati * ((hOmg) ** 2) / (2 * np.sqrt(A * mu))

KT = 1 * 10 ** (-6)
shift = A * (eE0 / hOmg) ** 2

Gamm = 0.005  # Gamma in eV.
hOmg = 0.5  # Photon energy eV

ek = A * (math.sqrt((0.1) ** 2 + (0.1) ** 2)) ** 2 + A * (eE0 / hOmg) ** 2

z = Gamm ** 2 + (ek - hOmg / 2 + hOmg * -0.5) ** 2

mlog = math.log(z)
nplog = np.log(z)

print('math.log result: ', mlog)
print('np.log result: ', nplog)

arrtest0 = [complex(0,mlog)]
arrtest1 = [complex(0,nplog)]

print('math.log result in array: ', arrtest0)
print('np.log result in array: ', arrtest1)

mlog = math.log(Gamm ** 2 + (ek - hOmg / 2 + hOmg * -0.5) ** 2)
nplog = np.log(Gamm ** 2 + (ek - hOmg / 2 + hOmg * -0.5) ** 2)

print('math.log result: ', mlog)
print('np.log result: ', nplog)

math.log result:  -2.0223879944493093
np.log result:  -2.0223879944493093
math.log result in array:  [-2.0223879944493093j]
np.log result in array:  [-2.0223879944493093j]
math.log result:  -2.0223879944493093
np.log result:  -2.0223879944493093


# Testing my Bessel function for various values of v and z

Ensuring that the number of terms makes the relative error equal to machine epsilon.

In [9]:
def my_Besselv(v, z):
    # WILL NOT WORK IF v IS NOT AN INTEGER
    # Conditional to handle case of negative v.
    if(v < 0):
        v = abs(v)
        resultsign = (-1) ** v
    else:
        resultsign = 1
    result = 0    
    # Loop to construct Bessel series sum.
    for n in range(0,20):
        sign = (-1)**n
        exp = 2 * n + v
        term = z ** exp
        r = n + v + 1
        if(r == 0):
            r = 1e-15
        denom = math.gamma(r)
        denom = denom * math.factorial(n)
        denom = denom * (2 ** exp)
        term = term / denom * sign
        # print('for ', n, ': ',term)
        result = result + term
        
    return result * resultsign

relerror = np.zeros(78)
l = 0

print('Comparing scipy library Bessel function and my Bessel function:')
print('================================================================================================')
print(' v | z | scipy result |   my result   | rel error')
print('================================================================================================')
for i in range(-3,3):
    for j in range(20,33):
        myres = my_Besselv(i, j/10)
        scires = scipy.special.jv(i, j/10)
        relerror[l] = abs((scires / myres) - 1)
        print('%3d|%3.1f|%14E|%15E|%E' % (i, j/10, scires, myres, relerror[l]))
        l = l + 1
print('================================================================================================')

avgerr = math.fsum(relerror) / l
print('Average relative error: ', avgerr)

Comparing scipy library Bessel function and my Bessel function:
 v | z | scipy result |   my result   | rel error
 -3|2.0| -1.289432E-01|  -1.289432E-01|2.220446E-16
 -3|2.1| -1.452767E-01|  -1.452767E-01|0.000000E+00
 -3|2.2| -1.623255E-01|  -1.623255E-01|2.220446E-16
 -3|2.3| -1.799789E-01|  -1.799789E-01|4.440892E-16
 -3|2.4| -1.981148E-01|  -1.981148E-01|2.220446E-16
 -3|2.5| -2.166004E-01|  -2.166004E-01|4.440892E-16
 -3|2.6| -2.352938E-01|  -2.352938E-01|2.220446E-16
 -3|2.7| -2.540453E-01|  -2.540453E-01|4.440892E-16
 -3|2.8| -2.726986E-01|  -2.726986E-01|2.220446E-16
 -3|2.9| -2.910926E-01|  -2.910926E-01|2.220446E-16
 -3|3.0| -3.090627E-01|  -3.090627E-01|2.220446E-16
 -3|3.1| -3.264428E-01|  -3.264428E-01|2.220446E-16
 -3|3.2| -3.430664E-01|  -3.430664E-01|3.330669E-16
 -2|2.0|  3.528340E-01|   3.528340E-01|0.000000E+00
 -2|2.1|  3.746236E-01|   3.746236E-01|1.110223E-16
 -2|2.2|  3.950587E-01|   3.950587E-01|2.220446E-16
 -2|2.3|  4.139146E-01|   4.139146E-01|2.220446E-16
 -